<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [47]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Izabel Sampaio Goes, 21000980' #@param {type:"string"}






In [48]:
import pandas as pd
import numpy as np

In [49]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [50]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [51]:
# prompt: exibir value_counts de host_id em ordem decrescente
df['host_id'].value_counts().head(10)


host_id
219517861    7
120762452    4
30283594     4
19303369     4
18996093     2
61391963     2
154258141    2
1513294      2
1475015      2
27973267     2
Name: count, dtype: int64

In [52]:
# prompt: dataframe com host_id, host_name e host_id value counts

# Assuming df is already defined as in the provided code.
host_summary = df.groupby(['host_id', 'host_name'])['host_id'].count().reset_index(name='count')
top_hosts = host_summary.sort_values('count', ascending=False).head()
top_hosts


host_id     host_name  count
713  219517861  Sonder (NYC)      7
394   30283594          Kara      4
605  120762452       Stanley      4
306   19303369        Hiroki      4
54     1513294       Urszula      2

Sonder (NYC),7;Kara,4;Stanley,4;Hiroki,4

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [53]:
# prompt: criar dataframe com id, host_id, host_name, price e uma coluna mais_de_um_imovel que deve ter valor bool indicando se aquele host_id se repete no dataframe original (df), ou seja, se a contagem daquele host_id é maior que um

# Assuming df is already defined as in the provided code.

# Create a new DataFrame with the desired columns
df_new = df[['id', 'host_id', 'host_name', 'price']].copy()

# Count the occurrences of each host_id
host_counts = df_new['host_id'].value_counts()

# Create the 'mais_de_um_imovel' column based on host_id counts
df_new['mais_de_um_imovel'] = df_new['host_id'].map(lambda x: host_counts[x] > 1)

# Display the new DataFrame
df_new


id    host_id    host_name  price  mais_de_um_imovel
0       20913      79402   Christiana    100              False
1       25235      87773       Shelly    125              False
2       76761     118971       Evelyn    169              False
3      135465     663879  Christopher    170              False
4      169002     805344         Alec     65              False
..        ...        ...          ...    ...                ...
799  36413779    2449230       Itamar     80              False
800  36421287  203866020        Karma     60              False
801  36439512  180336853         Dash     70              False
802  36452721  274079964      Richard    800              False
803  36472171   99144947       Brenda     80              False

[804 rows x 5 columns]

In [54]:
# prompt: Calcular a média de price considerando apenas linhas com False em mais_de_um_imovel e nomear essa variável preco_mais_um_imovel

preco_mais_um_imovel = df_new[df_new['mais_de_um_imovel'] == False]['price'].mean()
preco_mais_um_imovel

np.float64(150.34954007884363)

In [55]:
preco_mais_de_um_imovel = df_new[df_new['mais_de_um_imovel'] == True]['price'].mean()
preco_mais_de_um_imovel

np.float64(218.1627906976744)

In [56]:
...

print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD 67.81


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [57]:
# prompt: gerar dataframe apenas com linhas que contenham os outliers considerando os valores da coluna price

# Calculate the IQR for the 'price' column
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to include only outliers in 'price'
outliers_df = df[(df['price'] < lower_bound) | (df['price'] > upper_bound)]

# Filter for Manhattan outliers
manhattan_outliers = outliers_df[outliers_df['neighbourhood_group'] == 'Manhattan']

# Calculate the percentage of Manhattan outliers
percentage_manhattan_outliers = (len(manhattan_outliers) / len(df[df['neighbourhood_group'] == 'Manhattan'])) * 100
perc_Manhattan = percentage_manhattan_outliers

print(f'{percentage_manhattan_outliers:.2f}%')

# Display the DataFrame containing only outliers
outliers_df


10.05%


id                                               name    host_id  \
8      261781                  1500+ sq ft 2BR West Village Loft    1359519   
23     640589                     Sweet Super Bowl Accomodations    3198479   
26     692137              Stunning 3BR loft in Williamsburg!!!!    3538661   
28     725270        Very Large Loft in Chelsea for  Your Stay!!    3750402   
31     774899                 Unique West Village Loft with Deck     266210   
35     893413         Architecturally Stunning Former Synagogue!    4751930   
54    2252844                Great place to crash for Super Bowl   11505536   
73    3487445                   Sunny and Spacious 1 bedroom apt    2553182   
82    4000561                   Large apartment in Manhattan UWS    3325418   
88    4199411                          Greenwich Village Living!   21787420   
97    4537389                         New Years In New York City   22927481   
141   6589412                    Beautiful Park Slope Brownstone   34748949   
150   7294225  East Vlg 2 bdrm Private Apt - 1 Block to Union Sq   10353278   
151   7329781                               NoLita Design Studio   38398698   
174   8577949                   Large Victorian home in Brooklyn   11623096   
209  10123090                Midtown Manhattan steps from 5thAve   51948055   
219  10585031                 Luxury affordable in Williamsburg!   48988714   
268  13526990                       Historic Brooklyn Brownstone   45691473   
297  14488817                  Two bedrooms accommodate 4 guests    2766755   
308  15205915  2BdrmPRESIDENTIAL Luxury CONDO In the Heart of...   96098402   
317  15801420   Living excellence- deluxe 2 bedroom apt. in NYC!   30283594   
323  16098020  1 Bdrm PRESIDENTIAL RESERVE at Wyndham Midtown 45   69545883   
339  16487823  2 Double Bed Hotel Style WYNDHAM MIDTOWN 45 *NYC*   69545883   
365  17844682                    MIDTOWN * UNICORN * TOWNHOUSE!!   22456949   
396  19387694                                    Cool town house   14140453   
463  21950138      Beautiful 3 bedroom Brownstone in Boerum Hill   25321849   
464  21963398      4BR 2 Bath Williamsburg Apt with Private Roof   11310081   
470  22074303     1 BR w/Kit Wyndham 45 Midtown- Great Location!   21844834   
499  23293483                    20 w 53Rd st full furnished 2BR   53179388   
510  23995936  FLATIRON LUXURY HIGHRISE//STUNNING VIEWS//4 BE...  180548427   
563  27237591  *NO GUEST SERVICE FEE* One Bedroom Suite - Clo...  205031545   
574  27546818             Staying Good location to close US OPEN  142289373   
623  30193035  *NO GUEST SERVICE FEE* Beekman Tower Studio Su...  205031545   
645  30981245   Huge Artistic Downtown Loft - Designer Furniture   17981316   
646  30982372         Magnificent Industrial Chic 3 Bedroom Loft  228231975   
656  31444634  Wburg Duplex w/ Private Garden. Great for groups!   48392689   
677  32237493    Sonder | Stock Exchange | Serene 2BR + Sofa Bed  219517861   
680  32326698         Washington Suite at Incentra Village Hotel  241889662   
684  32476606  Recently Renovated & Furnished Apt- Room avail...  244008821   
691  32714435   Heart of NYC- deluxe 1BR apt with gorgeous views   30283594   
707  33580008    Sonder | Stock Exchange | Calming 3BR + Laundry  219517861   
708  33641209              Glamorous 3 Bedroom/3 Bath Brownstone  253528452   
718  34036047                                Harlem Time Capsule  256998898   
727  34312320    Sonder | The Biltmore | Stunning 1BR + Sofa Bed  219517861   
738  34859949   Amazing Townhouse Experience with Private Garden  262800168   
744  34997894                          entire apartment for rent  263670476   
755  35212471                  Bright & stylish 2BR in Manhattan  202805108   
778  35821347  TWO BEDROOM IN BEST PART OF BROOKLYN!! SLEEPS 6!!  262583834   
782  35884220             Luxury apartment in Manhattan midtown!  128175450   
795  36228909              Sunny SoHo Penthouse w/ Pvt. Terra

In [58]:
...

print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')


Percentuais de outliers de preços, Manhattan: 10.05 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [59]:
# prompt: Aplique lower em todas as strings da coluna name

# Assuming df is already defined as in the provided code.

def categorize_description(description):
    description = str(description).lower() # Convert to lowercase for case-insensitive matching
    if 'beautiful' in description:
        return 'beautiful'
    elif 'sunny' in description:
        return 'sunny'
    elif 'spacious' in description:
        return 'spacious'
    elif 'heart' in description:
        return 'heart'
    else:
        return 'others'

df['word'] = df['name'].apply(categorize_description)

# Group by 'word' and calculate the count and mean price
word_summary = df.groupby('word')['price'].agg(['count', 'mean']).reset_index()
word_summary


word  count        mean
0  beautiful     41  137.926829
1      heart     36  166.388889
2     others    624  157.886218
3   spacious     60  128.516667
4      sunny     43  137.674419

In [60]:
# prompt: Coloque os valores de word_summary no formato "beatiful,141.90;sunny,122.35", ou seja, "word,round(mean,2);word,round(mean,2)..."

word_summary_str = ""
for index, row in word_summary.iterrows():
    word_summary_str += f"{row['word']},{row['mean']:.2f};"

print(word_summary_str[:-1])


beautiful,137.93;heart,166.39;others,157.89;spacious,128.52;sunny,137.67


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [61]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [62]:
# prompt: Transformar NaN de df_taxes em zero (0)

# Fill NaN values in 'df_taxes' with 0
df_taxes = df_taxes.fillna(0)
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     0.0
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     0.0
1524  20325804  fines     0.0
1525  25872986  fines     0.0
1526  19365218  fines   906.0
1527   4235843  fines     0.0

[1528 rows x 3 columns]

In [63]:
# prompt: Faça um merge de df com df_taxes usando a coluna id de ambos

import pandas as pd
# Merge the two dataframes
df_merge = pd.merge(df, df_taxes, on='id', how='left')
df_merge

id                                               name    host_id  \
0        20913                 Charming 1 bed GR8 WBurg LOCATION!      79402   
1        20913                 Charming 1 bed GR8 WBurg LOCATION!      79402   
2        25235                  Large 2 Bedroom Great for Groups!      87773   
3        25235                  Large 2 Bedroom Great for Groups!      87773   
4        76761     Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm     118971   
...        ...                                                ...        ...   
1563  36439512                       The Bushwick Backhouse Loft.  180336853   
1564  36452721  Massage Spa. Stay overnight. Authors Artist dr...  274079964   
1565  36452721  Massage Spa. Stay overnight. Authors Artist dr...  274079964   
1566  36472171                      1 bedroom in sunlit apartment   99144947   
1567  36472171                      1 bedroom in sunlit apartment   99144947   

       host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0     Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1     Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
2         Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
3         Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
4         Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
...          ...                 ...                 ...       ...        ...   
1563        Dash            Brooklyn            Bushwick  40.70470  -73.92250   
1564     Richard            Brooklyn      Sheepshead Bay  40.59866  -73.95661   
1565     Richard            Brooklyn      Sheepshead Bay  40.59866  -73.95661   
1566      Brenda           Manhattan              Inwood  40.86845  -73.92449   
1567      Brenda           Manhattan              Inwood  40.86845  -73.92449   

            room_type  price  minimum_nights  number_of_reviews last_review  \
0     Entire home/apt    100               5                168  2018-07-22   
1     Entire home/apt    100               5                168  2018-07-22   
2     Entire home/apt    125              90                162  2019-06-28   
3     Entire home/apt    125              90                162  2019-06-28   
4     Entire home/apt    169               2                398  2019-06-28   
...               ...    ...             ...                ...         ...   
1563     Private room     70               2                  0         NaN   
1564     Private room    800               1                  0         NaN   
1565     Private room    800               1                  0         NaN   
1566     Private room     80               1                  0         NaN   
1567     Private room     80               1                  0         NaN   

      reviews_per_month  calculated_host_listings_count  availability_365  \
0                  1.57                               1                 0   
1                  1.57                               1                 0   
2                  1.46                               2               137   
3                  1.46                               2               137   
4                  3.97                               3               182   
...                 ...                             ...               ...   
1563                NaN                               1               255   
1564                NaN                               1                23   
1565                NaN                               1                23   
1566                NaN                               1                79   
1567                NaN                               1                79   

        word   type   value  
0     others  taxes   789.0  
1     others  fines   796.0  
2     others  taxes  1779.0  
3     others  fines     0.0  
4     others  taxes  1516.0  
...      ...    ...     

In [67]:
# prompt: value_counts de neighbourhood_group

# Assuming df is already defined as in the provided code.
df['neighbourhood_group'].value_counts()


neighbourhood_group
Manhattan        368
Brooklyn         330
Queens            79
Bronx             20
Staten Island      7
Name: count, dtype: int64

In [76]:
# prompt: Considerando df_merge, criar dataframe com média de valores da coluna value para taxes e fines da coluna type para cada valor distinto da coluna neighbourhood_group

# Group by 'neighbourhood_group' and 'type', then calculate the mean of 'value'
result_df = df_merge.groupby(['neighbourhood_group', 'type'])['value'].mean().unstack()

# Display the resulting DataFrame
result_df


type                      fines        taxes
neighbourhood_group                         
Bronx                126.473684   851.052632
Brooklyn             314.472313   858.276873
Manhattan            314.980226   888.844633
Queens               322.769231   804.166667
Staten Island        581.333333  1157.500000

In [78]:
result_df['fines'] = result_df['fines'].astype(int)
result_df['taxes'] = result_df['taxes'].astype(int)
result_df

type                 fines  taxes
neighbourhood_group              
Bronx                  126    851
Brooklyn               314    858
Manhattan              314    888
Queens                 322    804
Staten Island          581   1157

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [100]:
# prompt: Ordenar df por neighbourhood_group, neighbourhood

import pandas as pd
# Assuming df_merge is already defined as in the provided code.

df_report = df[['neighbourhood_group', 'neighbourhood', 'price']].copy()
df_report = df_report.sort_values(by=['neighbourhood_group', 'neighbourhood'])
df_report

neighbourhood_group      neighbourhood  price
442               Bronx            Belmont     29
769               Bronx  Claremont Village     67
353               Bronx          Concourse     40
597               Bronx           Edenwald     90
625               Bronx           Edenwald     80
..                  ...                ...    ...
495       Staten Island        South Beach     20
607       Staten Island        South Beach    100
560       Staten Island         St. George    130
714       Staten Island          Todt Hill    135
408       Staten Island      Tompkinsville     59

[804 rows x 3 columns]

In [101]:
# prompt: criar dataframe df_report_mean com base em df_report e adicionando uma linha para cada neighbourhood_group com valor 'mean' em neighbourhood e a média daquele neighbourhood_group em price

import pandas as pd
# Calculate the mean price for each neighbourhood_group
mean_prices = df_report.groupby('neighbourhood_group')['price'].mean()

# Create an empty list to store the rows of the new DataFrame
rows = []

# Iterate over the unique neighbourhood_groups
for group in df_report['neighbourhood_group'].unique():
    # Create a new row for the mean price of the neighbourhood_group
    rows.append({'neighbourhood_group': group, 'neighbourhood': 'mean', 'price': mean_prices[group]})

# Create the new DataFrame
df_report_mean = pd.DataFrame(rows)

# Concatenate the original and new DataFrames
df_report = pd.concat([df_report, df_report_mean], ignore_index=True)
df_report

neighbourhood_group      neighbourhood       price
0                 Bronx            Belmont   29.000000
1                 Bronx  Claremont Village   67.000000
2                 Bronx          Concourse   40.000000
3                 Bronx           Edenwald   90.000000
4                 Bronx           Edenwald   80.000000
..                  ...                ...         ...
804               Bronx               mean   71.500000
805            Brooklyn               mean  128.087879
806           Manhattan               mean  196.157609
807              Queens               mean   92.227848
808       Staten Island               mean   89.428571

[809 rows x 3 columns]

In [103]:
# prompt: Ordenar df_report por neighbourhood_group

df_report = df_report.sort_values(by=['neighbourhood_group'])
df_report


neighbourhood_group  neighbourhood       price
0                 Bronx        Belmont   29.000000
10                Bronx    Morris Park   40.000000
2                 Bronx      Concourse   40.000000
3                 Bronx       Edenwald   90.000000
4                 Bronx       Edenwald   80.000000
..                  ...            ...         ...
802       Staten Island      Todt Hill  135.000000
800       Staten Island    South Beach  100.000000
803       Staten Island  Tompkinsville   59.000000
801       Staten Island     St. George  130.000000
808       Staten Island           mean   89.428571

[809 rows x 3 columns]

In [109]:
# prompt: criar coluna line_id em df_report que vai de 1 a 30, começando na primeira linha. Ao chegar no 30, a próxima linha deve voltar a contagem ao 1

# Create the 'line_id' column
line_ids = []
current_line_id = 1
for i in range(len(df_report)):
    line_ids.append(current_line_id)
    current_line_id += 1
    if current_line_id > 30:
        current_line_id = 1
df_report['line_id'] = line_ids
df_report


neighbourhood_group  neighbourhood       price  line_id  page_id
0                 Bronx        Belmont   29.000000        1        1
10                Bronx    Morris Park   40.000000        2        1
2                 Bronx      Concourse   40.000000        3        1
3                 Bronx       Edenwald   90.000000        4        1
4                 Bronx       Edenwald   80.000000        5        1
..                  ...            ...         ...      ...      ...
802       Staten Island      Todt Hill  135.000000       25       29
800       Staten Island    South Beach  100.000000       26       29
803       Staten Island  Tompkinsville   59.000000       27       29
801       Staten Island     St. George  130.000000       28       29
808       Staten Island           mean   89.428571       29       29

[809 rows x 5 columns]

In [111]:
# prompt: criar coluna page_id em df_report que inicia em 1 e, a cada próximo 1, é acrescido em 1

# Assuming df_report is already defined as in the provided code.

# Create the 'page_id' column
page_ids = []
current_page_id = 1
current_line_count = 0

for index, row in df_report.iterrows():
    page_ids.append(current_page_id)
    current_line_count += 1
    if current_line_count == 30:
        current_page_id += 1
        current_line_count = 0

df_report['page_id'] = page_ids
df_report.head(35)


neighbourhood_group       neighbourhood   price  line_id  page_id
0                 Bronx             Belmont    29.0        1        1
10                Bronx         Morris Park    40.0        2        1
2                 Bronx           Concourse    40.0        3        1
3                 Bronx            Edenwald    90.0        4        1
4                 Bronx            Edenwald    80.0        5        1
5                 Bronx           Fieldston    85.0        6        1
6                 Bronx           Fieldston   109.0        7        1
7                 Bronx             Fordham    79.0        8        1
8                 Bronx          Highbridge    22.0        9        1
9                 Bronx             Melrose    69.0       10        1
1                 Bronx   Claremont Village    67.0       11        1
11                Bronx          Mott Haven   100.0       12        1
12                Bronx         Port Morris    28.0       13        1
13                Bronx           Riverdale    99.0       14        1
14                Bronx            Van Nest   107.0       15        1
15                Bronx           Wakefield   120.0       16        1
16                Bronx           Wakefield   120.0       17        1
17                Bronx      Williamsbridge    45.0       18        1
18                Bronx      Williamsbridge    65.0       19        1
804               Bronx                mean    71.5       20        1
19                Bronx      Williamsbridge    36.0       21        1
24             Brooklyn  Bedford-Stuyvesant    59.0       22        1
23             Brooklyn  Bedford-Stuyvesant   250.0       23        1
185            Brooklyn       Cypress Hills    75.0       24        1
21             Brooklyn           Bay Ridge  1800.0       25        1
20             Brooklyn           Bay Ridge   100.0       26        1
25             Brooklyn  Bedford-Stuyvesant    45.0       27        1
22             Brooklyn  Bedford-Stuyvesant   125.0       28        1
26             Brooklyn  Bedford-Stuyvesant    55.0       29        1
31             Brooklyn  Bedford-Stuyvesant   100.0       30        1
28             Brooklyn  Bedford-Stuyvesant    59.0        1        2
29             Brooklyn  Bedford-Stuyvesant   150.0        2        2
30             Brooklyn  Bedford-Stuyvesant    60.0        3        2
32             Brooklyn  Bedford-Stuyvesant    50.0        4        2
33             Brooklyn  Bedford-Stuyvesant   150.0        5        2

In [112]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 15668


#### Aqui um *sample* do report.

In [113]:
df_report.iloc[-10::,:]

neighbourhood_group    neighbourhood       price  line_id  page_id
760              Queens  Jackson Heights   50.000000       20       27
787              Queens        Sunnyside   50.000000       21       27
797       Staten Island    Arden Heights   83.000000       22       27
798       Staten Island    Randall Manor   99.000000       23       27
799       Staten Island      South Beach   20.000000       24       27
802       Staten Island        Todt Hill  135.000000       25       27
800       Staten Island      South Beach  100.000000       26       27
803       Staten Island    Tompkinsville   59.000000       27       27
801       Staten Island       St. George  130.000000       28       27
808       Staten Island             mean   89.428571       29       27